In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)


---------------------------------------------------------------

root: c:\SESA\dev\bulletin
parent: c:\SESA\dev
default_input: c:\SESA\dev\input
default_output: c:\SESA\dev\output

---------------------------------------------------------------



In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
notifica = Notifica()
notifica.load()
notifica.df

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc
0,66722,2021-06-12,2021-06-12,1.0,1.0,2021-06-12,None,2.0,1.0,3.0,2021-06-15,3.0,702509344393636,14892374962,NaN,0.0,HELOISA APARECIDA FERNANDES DA SILVA,2.0,2019-03-28,CAROLINE APARECIDA ANTONIO DA COSTA,99.0,5.0,,NaN,NaN,1.0,,41.0,411520.0,8.70434e+07,Rua Rio Amazonas,Conjunto Residencial Paulino Carlos Filho,41.0,412625.0,2021-06-11,2021-06-11,2021-06-11,2021-06-11,-1.000000e+00,12.0,167.0,2.0,FSJ,2021-06-05,2021-06-11,2.0,HELOISAAPARECIDAFERNANDESDASILVA2411520,HELOISAAPARECIDAFERNANDESDASILVA2412625,NILDACASTORINACORREIACRUZ45412710,NILDACASTORINACORREIACRUZ47412710,HELOISAAPARECIDAFERNANDESDASILVACAROLINEAPAREC...,HELOISAAPARECIDAFERNANDESDASILVA28032019
1,66776,2021-06-12,2021-06-16,2.0,1.0,2021-06-16,None,2.0,1.0,1.0,2021-06-03,3.0,704804585635947,85675750959,NaN,0.0,ANTONIO DE FRANCA ROCHA,1.0,1971-05-27,MARIA SANTOS ROCHA,3.0,5.0,,NaN,NaN,1.0,,41.0,410580.0,8.34051e+07,Rua Getúlio Vargas,Fátima,41.0,999999.0,2021-06-12,2021-06-03,2020-04-17,2020-04-19,-1.000000e+00,12.0,167.0,2.0,FARMACIA DESCONTAO,2021-05-31,2021-06-03,50.0,ANTONIODEFRANCAROCHA50410580,ANTONIODEFRANCAROCHA50999999,ELIANADASILVA42411810,ELIANADASILVA44411810,ANTONIODEFRANCAROCHAMARIASANTOSROCHA,ANTONIODEFRANCAROCHA27051971
2,66777,2021-06-12,2021-03-23,1.0,1.0,2021-06-12,None,2.0,1.0,3.0,2020-12-16,3.0,702002811670085,07901900954,NaN,NaN,EVANDRA PEREIRA DA SILVA,2.0,1986-04-02,ANTONIA SALETE FRANCESCHI DA SILVA,1.0,5.0,auxiliar de costura,7632.0,NaN,1.0,,41.0,412380.0,8.565e+07,carvalho,santo antonio,41.0,412380.0,2021-06-12,2021-06-12,NaT,2021-06-11,2.108010e+11,12.0,167.0,2.0,,2021-06-09,2021-06-12,35.0,EVANDRAPEREIRADASILVA35412380,EVANDRAPEREIRADASILVA35412380,SALETELOURDESREBESCHINIDASILVA66412860,SALETELOURDESREBESCHINIDASILVA68412860,EVANDRAPEREIRADASILVAANTONIASALETEFRANCESCHIDA...,EVANDRAPEREIRADASILVA02041986
3,66824,2021-06-12,2021-06-10,1.0,1.0,2021-06-12,None,2.0,1.0,3.0,2020-04-26,3.0,703402234751018,04125881960,NaN,0.0,CARMEM FERREIRA MENDES,2.0,1968-08-09,LEONINA LEODORO DA SILVA,99.0,5.0,,NaN,NaN,1.0,,41.0,411520.0,8.6975e+07,DR CAETANO MUNHOS DA ROCHA,CENTRO,41.0,411750.0,NaT,2021-06-10,NaT,2021-06-10,-1.000000e+00,12.0,167.0,2.0,,2021-06-09,2021-06-12,52.0,CARMEMFERREIRAMENDES52411520,CARMEMFERREIRAMENDES52411750,MARCIANOPEREIRALUCAS33410510,MARCIANOPEREIRALUCAS35410510,CARMEMFERREIRAMENDESLEONINALEODORODASILVA,CARMEMFERREIRAMENDES09081968
4,66864,2021-06-12,2021-06-14,1.0,1.0,2021-06-12,None,2.0,1.0,3.0,NaT,3.0,703400355043500,07298899988,NaN,0.0,ANDERSON MOREIRA,1.0,1990-08-08,MARLENE APARECIDA FARIAS MOREIRA,99.0,0.0,,0.0,NaN,1.0,,41.0,410580.0,8.34016e+07,Rua do Nambu,Arruda,41.0,410580.0,2021-06-12,2021-06-02,2021-06-14,2021-06-14,-1.000000e+00,12.0,167.0,2.0,FARMACIAS UNIPRECO,2021-06-08,2021-06-02,30.0,ANDERSONMOREIRA30410580,ANDERSONMOREIRA30410580,VALDOMIRADASILVACORDEIRO83412625,VALDOMIRADASILVACORDEIRO85412625,ANDERSONMOREIRAMARLENEAPARECIDAFARIASMOREIRA,ANDERSONMOREIRA08081990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178761,4266621,2021-06-16,NaT,1.0,1.0,2021-06-16,None,2.0,1.0,3.0,NaT,3.0,709608675395674,09373551000,NaN,NaN,DILMO PEDROLLO,1.0,1949-11-18,IZABEL PEDROLLO,99.0,NaN,,NaN,NaN,1.0,,41.0,411580.

In [4]:
id_com_idades = pd.read_excel('id_com_idades.xlsx')
id_com_idades = id_com_idades.rename(columns={'idade':'idade_temp'})

In [5]:
casos_pr = notifica.df.loc[notifica.df['uf_residencia'] == 41, ['id', 'evolucao','data_cura_obito','data_diagnostico', 'sexo','raca_cor','ibge_residencia','idade','hash','hash_mae','hash_nasc']]
casos_pr

,id,evolucao,data_cura_obito,data_diagnostico,sexo,raca_cor,ibge_residencia,idade,hash,hash_mae,hash_nasc
0,66722,3.0,2021-06-15,2021-06-11,2.0,99.0,411520.0,2.0,HELOISAAPARECIDAFERNANDESDASILVA2411520,HELOISAAPARECIDAFERNANDESDASILVACAROLINEAPAREC...,HELOISAAPARECIDAFERNANDESDASILVA28032019
1,66776,1.0,2021-06-03,2021-06-03,1.0,3.0,410580.0,50.0,ANTONIODEFRANCAROCHA50410580,ANTONIODEFRANCAROCHAMARIASANTOSROCHA,ANTONIODEFRANCAROCHA27051971
2,66777,3.0,2020-12-16,2021-06-12,2.0,1.0,412380.0,35.0,EVANDRAPEREIRADASILVA35412380,EVANDRAPEREIRADASILVAANTONIASALETEFRANCESCHIDA...,EVANDRAPEREIRADASILVA02041986
3,66824,3.0,2020-04-26,2021-06-12,2.0,99.0,411520.0,52.0,CARMEMFERREIRAMENDES52411520,CARMEMFERREIRAMENDESLEONINALEODORODASILVA,CARMEMFERREIRAMENDES09081968
4,66864,3.0,NaT,2021-06-02,1.0,99.0,410580.0,30.0,ANDERSONMOREIRA30410580,ANDERSONMOREIRAMARLENEAPARECIDAFARIASMOREIRA,ANDERSONMOREIRA08081990
...,...,...,...,...,...,...,...,...,...,...,...
1178761,4266621,3.0,NaT,2021-06-16,1.0,99.0,411580.0,71.0,DILMOPEDROLLO71411580,DILMOPEDROLLOIZABELPEDROLLO,DILMOPEDROLLO18111949
1178762,4266635,3.0,NaT,2021-06-15,1.0,99.0,411990.0,54.0,JOSECARLOSDOCARMO54411990,JOSECARLOSDOCARMOGLACIVALENTIMDOCARMO,JOSECARLOSDOCARMO27091966
1178763,4266643,1.0,2021-06-16,2021-06-16,2.0,99.0,412790.0,43.0,ROSELAINEGONCALVESDEOLIVEIRA43412790,ROSELAINEGONCALVESDEOLIVEIRAROSABELIODEOLIVEIRA,ROSELAINEGONCALVESDEOLIVEIRA19021978
1178764,4266662,1.0,2021-06-16,2021-06-04,2.0,99.0,410320.0,46.0,RITAMARABERTUCCI46410320,RITAMARABERTUCCINAIRPEREIRABERTUCCI,RITAMARABERTUCCI12101974


In [6]:
casos_pr = casos_pr.merge(how = 'left', on = 'id', right = id_com_idades)
casos_pr = casos_pr.drop(index=casos_pr.loc[casos_pr['id'] == 2196181].index)
casos_pr.loc[casos_pr['idade'] == -99, 'idade'] = casos_pr.loc[casos_pr['idade'] == -99, 'idade_temp']
casos_pr = casos_pr[['id', 'evolucao','data_cura_obito','data_diagnostico', 'sexo','raca_cor','ibge_residencia','idade','hash','hash_mae','hash_nasc']]

In [7]:
casos_pr = casos_pr.drop_duplicates('hash_mae', keep='first')

In [8]:
casos_pr = casos_pr.drop_duplicates('hash_nasc', keep='first')

In [9]:
casos_pr = casos_pr.loc[casos_pr['idade'] < 20]

casos_pr.loc[casos_pr['evolucao'] == 1, 'evolucao'] = 'Cura'
casos_pr.loc[casos_pr['evolucao'] == 2, 'evolucao'] = 'Óbito por COVID'
casos_pr.loc[casos_pr['evolucao'] == 3, 'evolucao'] = 'Não se aplica'
casos_pr.loc[casos_pr['evolucao'] == 4, 'evolucao'] = 'Óbito por outras causas'

casos_pr.loc[casos_pr['raca_cor'] == 1, 'raca_cor'] = 'Branca'
casos_pr.loc[casos_pr['raca_cor'] == 2, 'raca_cor'] = 'Preta'
casos_pr.loc[casos_pr['raca_cor'] == 3, 'raca_cor'] = 'Parda'
casos_pr.loc[casos_pr['raca_cor'] == 4, 'raca_cor'] = 'Amarela'
casos_pr.loc[casos_pr['raca_cor'] == 5, 'raca_cor'] = 'Indígena'
casos_pr.loc[casos_pr['raca_cor'] == 99, 'raca_cor'] = 'Ignorado'



casos_pr.loc[casos_pr['sexo'] == 1, 'sexo'] = 'M'
casos_pr.loc[casos_pr['sexo'] == 2, 'sexo'] = 'F'
casos_pr.loc[casos_pr['sexo'] == 3, 'sexo'] = 'Não informado'

In [10]:
casos_pr

,id,evolucao,data_cura_obito,data_diagnostico,sexo,raca_cor,ibge_residencia,idade,hash,hash_mae,hash_nasc
0,66722,Não se aplica,2021-06-15,2021-06-11,F,Ignorado,411520.0,2.0,HELOISAAPARECIDAFERNANDESDASILVA2411520,HELOISAAPARECIDAFERNANDESDASILVACAROLINEAPAREC...,HELOISAAPARECIDAFERNANDESDASILVA28032019
10,66942,Não se aplica,2020-04-11,2021-06-12,M,Ignorado,410360.0,2.0,MIGUELBENJAMINMENDES2410360,MIGUELBENJAMINMENDESANAPAULADASILVA,MIGUELBENJAMINMENDES06052019
14,67024,Não se aplica,NaT,2021-06-12,M,Ignorado,410780.0,8.0,HIAGOMENDONCABARBOZA8410780,HIAGOMENDONCABARBOZAVANESSADASILVAMENDONCA,HIAGOMENDONCABARBOZA17012013
16,67141,Não se aplica,NaT,2021-06-12,M,Ignorado,410940.0,5.0,LORENZOEMANUELMATOSDASILVA5410940,LORENZOEMANUELMATOSDASILVAKENDRATHALIADEALMEID...,LORENZOEMANUELMATOSDASILVA25042016
19,67202,Não se aplica,NaT,2021-06-12,F,Ignorado,411960.0,18.0,MIRIANIDOSSANTOS18411960,MIRIANIDOSSANTOSROSADOSSANTOS,MIRIANIDOSSANTOS15102002
...,...,...,...,...,...,...,...,...,...,...,...
1173898,4266256,Não se aplica,NaT,2021-05-26,M,Ignorado,411940.0,14.0,JOAOPEDROMILEOMAINARDES14411940,JOAOPEDROMILEOMAINARDESCELINAMARIACOBBEMILEO,JOAOPEDROMILEOMAINARDES03102006
1173899,4266261,Não se aplica,NaT,2021-05-27,F,Ignorado,411575.0,3.0,RAYANEORTIZFERNANDES3411575,RAYANEORTIZFERNANDESMARINALVAORTIZFRANCA,RAYANEORTIZFERNANDES14022018
1173907,4266294,Não se aplica,NaT,2021-06-16,M,Ignorado,411240.0,19.0,VINICIUSALBERICOMARTINS19411240,VINICIUSALBERICOMARTINSANGELITAALBERICOMARTINS,VINICIUSALBERICOMARTINS02062002
1173916,4266339,Não se aplica,NaT,2021-06-16,M,Ignorado,410610.0,15.0,ELIABEMELODESOUZA15410610,ELIABEMELODESOUZALUCIANEDEMELODESOUZA,ELIABEMELODESOUZA14072005


In [11]:
casos_pr['ibge_residencia'].isna().values.any()

False

In [12]:
casos_pr = casos_pr[casos_pr['data_diagnostico'].notnull()]

In [13]:
casos_pr.loc[casos_pr['evolucao'] == 'Óbito por COVID', 'OBITO'] = 'SIM'

In [14]:
casos_pr = casos_pr.drop(columns={'hash', 'hash_mae', 'hash_nasc'})
casos_pr['ibge_residencia'] = casos_pr['ibge_residencia'].fillna(0).astype('int')
casos_pr['idade'] = casos_pr['idade'].fillna(0).astype('int')

In [15]:
casos_pr['ibge_residencia'] = casos_pr['ibge_residencia'].fillna(0).astype('int')
casos_pr['idade'] = casos_pr['idade'].fillna(0).astype('int')

In [16]:
casos_pr.loc[(casos_pr['idade'] >= 0) & (casos_pr['idade'] < 10), 'faixa_et1'] = '0-9'
casos_pr.loc[(casos_pr['idade'] >= 10) & (casos_pr['idade'] < 20), 'faixa_et1'] = '10-19'

casos_pr.loc[(casos_pr['idade'] >= 10) & (casos_pr['idade'] <= 13), 'faixa_et2'] = '10-13'
casos_pr.loc[(casos_pr['idade'] >= 14) & (casos_pr['idade'] <= 16), 'faixa_et2'] = '14-16'
casos_pr.loc[(casos_pr['idade'] >= 17) & (casos_pr['idade'] <= 19), 'faixa_et2'] = '17-19'

In [17]:
casos_pr['faixa_et2'] = casos_pr['faixa_et2'].fillna('Não se aplica')

In [18]:
casos_pr.loc[casos_pr['idade'] == -99]

,id,evolucao,data_cura_obito,data_diagnostico,sexo,raca_cor,ibge_residencia,idade,OBITO,faixa_et1,faixa_et2


In [19]:
casos_pr.to_excel('casos_pr_adolescentes.xlsx', index=False)